# Different feature selection methods experiments
1) Traditional - PCA  
2) Filter - Mutual Information  
3) Wrapper - Recursive Feature Elimination for GradientBoosting Regression / Random Forest Regression  
4) Embedded  
　　－　ElasticNet  
　　－　ElasticNetCV  
　　－　RandomForest  
　　－　MultiTaskElasticNetCV
  
### Final pick: ElasticNetCV

### Reasons for choosing ElasticNetCV: 
1) Effrosynidis and Arampatzis (2021) has compared different feature selection methods in environmental variables, and found the embedded methods and ensembled methods outperform filter and wrapper methods considering performance, stability and time consuming. In that paper, embedded Random Forest was used, while in my experiment, ElasticNetCV outperforms RF.
https://www.sciencedirect.com/science/article/pii/S1574954121000157

2) Also, Pirhooshyaran and Snyder (2020) has used ElasticNet to do the Feature selection.
https://www.sciencedirect.com/science/article/pii/S0029801820304492
      
3) However, Meyer et al. (2018) emphasized the importance of cross-validation.Feature Selection in conjunction with target-oriented CV can reduce over-fitting.  
https://www.sciencedirect.com/science/article/pii/S1364815217310976

4) Compared with other methods (Traditiona, Filter, Wrapper, RF, MultiTaskElasticNet), ElasticNetCV is quite fast.

Will do more comparision of feature selection methods combined with prediction if necessary.

In [9]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import sys
import datetime
import time

In [3]:
import pickle
def save(fname, data, protocol=3):
    with open(fname, "wb") as f:
        pickle.dump(data, f, protocol)

def load(fname):
    with open(fname, "rb") as f:
        return pickle.load(f)

In [74]:
table = load('Imputed_Measurement')
table.columns = ['1Wave Height (m)', '1Max Wave Height (m)', '1Tpeak (s)', '1Tz (s)',
       '1Peak Direction (degrees)', '1Spread (degrees)', '1Sea Temp (C)',
       '2Wave Height (m)', '2Max Wave Height (m)', '2Tpeak (s)', '2Tz (s)',
       '2Peak Direction (degrees)', '2Spread (degrees)', '2Sea Temp (C)',
       '3Wave Height (m)', '3Max Wave Height (m)', '3Tpeak (s)', '3Tz (s)',
       '3Peak Direction (degrees)', '3Spread (degrees)', '3Sea Temp (C)']
table

,1Wave Height (m),1Max Wave Height (m),1Tpeak (s),1Tz (s),1Peak Direction (degrees),1Spread (degrees),1Sea Temp (C),2Wave Height (m),2Max Wave Height (m),2Tpeak (s),...,2Peak Direction (degrees),2Spread (degrees),2Sea Temp (C),3Wave Height (m),3Max Wave Height (m),3Tpeak (s),3Tz (s),3Peak Direction (degrees),3Spread (degrees),3Sea Temp (C)
Time (GMT),,,,,,,,,,,,,,,,,,,,,
2010-01-02 00:00:00,0.41,0.53,10.5,6.8,188.0,16.0,8.1,0.58,0.77,13.3,...,201.0,15.0,9.0,0.52,0.94,10.5,4.0,283.0,27.0,9.0
2010-01-02 00:30:00,0.37,0.59,11.1,6.3,187.0,10.0,8.2,0.52,0.79,11.1,...,208.0,19.0,9.1,0.54,0.79,4.8,4.1,316.0,42.0,9.1
2010-01-02 01:00:00,0.38,0.55,12.5,6.8,184.0,9.0,8.1,0.52,0.84,11.1,...,215.0,19.0,9.0,0.53,0.74,10.0,4.2,280.0,31.0,9.0
2010-01-02 01:30:00,0.38,0.59,11.8,7.0,184.0,12.0,8.2,0.51,0.81,12.5,...,203.0,14.0,9.0,0.56,0.80,4.4,4.1,326.0,49.0,9.0
2010-01-02 02:00:00,0.39,0.51,11.1,7.3,190.0,11.0,8.3,0.48,0.63,10.5,...,212.0,15.0,9.0,0.57,0.89,10.0,4.2,290.0,22.0,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30 21:30:00,0.72,1.23,5.0,4.3,184.0,19.0,10.5,0.89,1.08,10.0,...,205.0,14.0,10.9,1.27,1.96,11.8,6.6,290.0,28.0,10.6
2019-12-30 22:00:00,0.71,1.03,5.3,4.3,186.0,23.0,10.5,0.85,1.40,11.1,...,211.0,17.0,10.9,1.30,1.98,11.1,6.8,274.0,34.0,10.6
2019-12-30 22:30:00,0.71,0.92,5.6,4.2,181.0,19.0,10.5,0.81,1.19,11.8,...,210.0,13.0,10.9,1.16,1.76,11.8,6.5,277.0,40.0,10.6


In [207]:
# Dataset normalization
from sklearn.preprocessing import MinMaxScaler
table_scale = table.copy()
scalers={}
for i in table.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(table_scale[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    table_scale[i]=s_s
table_scale

,1Wave Height (m),1Max Wave Height (m),1Tpeak (s),1Tz (s),1Peak Direction (degrees),1Spread (degrees),1Sea Temp (C),2Wave Height (m),2Max Wave Height (m),2Tpeak (s),...,2Peak Direction (degrees),2Spread (degrees),2Sea Temp (C),3Wave Height (m),3Max Wave Height (m),3Tpeak (s),3Tz (s),3Peak Direction (degrees),3Spread (degrees),3Sea Temp (C)
Time (GMT),,,,,,,,,,,,,,,,,,,,,
2010-01-02 00:00:00,-0.932920,-0.840781,-0.263140,-0.426901,-0.094515,-0.453720,-0.568421,-0.860722,-0.924542,-0.131263,...,0.080766,-0.604626,-0.314010,-0.892063,-0.876021,-0.345725,-0.801980,0.453322,-0.384129,-0.444749
2010-01-02 00:30:00,-0.938113,-0.838048,-0.221268,-0.485380,-0.099070,-0.590006,-0.557895,-0.875383,-0.923584,-0.293928,...,0.116152,-0.507376,-0.304348,-0.887831,-0.883057,-0.769517,-0.792079,0.607044,0.015139,-0.432294
2010-01-02 01:00:00,-0.936815,-0.839870,-0.123567,-0.426901,-0.112735,-0.612720,-0.568421,-0.875383,-0.921189,-0.293928,...,0.151538,-0.507376,-0.314010,-0.889947,-0.885402,-0.382900,-0.782178,0.439347,-0.277658,-0.444749
2010-01-02 01:30:00,-0.936815,-0.838048,-0.172417,-0.403509,-0.112735,-0.544577,-0.557895,-0.877826,-0.922626,-0.190414,...,0.090876,-0.628939,-0.314010,-0.883598,-0.882588,-0.799257,-0.792079,0.653626,0.201464,-0.444749
2010-01-02 02:00:00,-0.935517,-0.841693,-0.221268,-0.368421,-0.085405,-0.567292,-0.547368,-0.885156,-0.931250,-0.338292,...,0.136372,-0.604626,-0.314010,-0.881481,-0.878366,-0.382900,-0.782178,0.485929,-0.517219,-0.469658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30 21:30:00,-0.892678,-0.808895,-0.646967,-0.719298,-0.112735,-0.385577,-0.315789,-0.784974,-0.909690,-0.375261,...,0.100987,-0.628939,-0.130435,-0.733333,-0.828183,-0.249071,-0.544554,0.485929,-0.357511,-0.245473
2019-12-30 22:00:00,-0.893976,-0.818005,-0.626031,-0.719298,-0.103625,-0.294719,-0.315789,-0.794748,-0.894359,-0.293928,...,0.131317,-0.556001,-0.130435,-0.726984,-0.827245,-0.301115,-0.524752,0.411398,-0.197804,-0.245473
2019-12-30 22:30:00,-0.893976,-0.823016,-0.605095,-0.730994,-0.126400,-0.385577,-0.315789,-0.804522,-0.904420,-0.242171,...,0.126262,-0.653251,-0.130435,-0.756614,-0.837563,-0.249071,-0.554455,0.425372,-0.038097,-0.245473


# Traditional - PAC

In [13]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(table, table.iloc[:,0])
# print(pca.explained_variance_ratio_)
# print(pca.explained_variance_)
a = 0
for i in np.array(-pca.explained_variance_ratio_).argsort():
    a = a + (pca.explained_variance_ratio_/pca.explained_variance_ratio_.sum())[i]
    print(a,' ',table.keys()[i])

0.5386651029993477   1Wave Height (m)
0.7173577380193081   1Max Wave Height (m)
0.8776476929773849   1Tpeak (s)
0.9319152643474427   1Tz (s)
0.9585270589211612   1Peak Direction (degrees)
0.9734655236285388   1Spread (degrees)
0.9858884848490796   1Sea Temp (C)
0.9918772442759671   2Wave Height (m)
0.9945346426192438   2Max Wave Height (m)
0.9968432810363426   2Tpeak (s)
0.9980290506912864   2Tz (s)
0.9985864912100719   2Peak Direction (degrees)
0.9989567699817672   2Spread (degrees)
0.9993119708636311   2Sea Temp (C)
0.9996246112691141   3Wave Height (m)
0.999753719501879   3Max Wave Height (m)
0.9998466063738439   3Tpeak (s)
0.9999315118846576   3Tz (s)
0.9999796141544534   3Peak Direction (degrees)
0.999993629925608   3Spread (degrees)
1.0   3Sea Temp (C)


# Filter - Mutual Information 

In [204]:
X, y = table.values[:-12], table.iloc[12:,3].values
# X, y = table_scale.values, table_scale.iloc[:,0].values

In [50]:
from sklearn.feature_selection import VarianceThreshold,SelectKBest
from sklearn.feature_selection import mutual_info_regression
sk1 = SelectKBest(mutual_info_regression,k=21)
sk1.fit(X, y)
print(sk1)
print(sk1.scores_)
print(sk1.transform(X))

# print(pca.explained_variance_ratio_)
# print(pca.explained_variance_)
t0 = time.time()
a = 0
for i in np.array(-sk1.scores_).argsort():
    a = a + (sk1.scores_/sk1.scores_.sum())[i]
    print(a,' ',table.keys()[i])
print(time.time()-t0)

SelectKBest(k=21,
            score_func=<function mutual_info_regression at 0x000002247D0AC048>)
[5.07405018 1.81464604 0.15968596 0.21214894 0.11036467 0.21475547
 0.11472302 1.04525642 1.00223748 0.09298029 0.1608925  0.06613631
 0.12693138 0.10484305 0.15656309 0.14568622 0.11694322 0.14641247
 0.04043402 0.04073263 0.12529234]
[[-0.93292031 -0.84078145 -0.26314009 ...  0.45332178 -0.38412926
  -0.44474887]
 [-0.93811284 -0.83804829 -0.22126805 ...  0.60704364  0.0151388
  -0.43229414]
 [-0.93681471 -0.8398704  -0.1235666  ...  0.43934707 -0.27765778
  -0.44474887]
 ...
 [-0.89397631 -0.82301593 -0.60509515 ...  0.42537235 -0.03809694
  -0.24547316]
 [-0.88878378 -0.80798356 -0.60509515 ...  0.45798002 -0.27765778
  -0.24547316]
 [-0.89267818 -0.81709409 -0.60509515 ...  0.41139764 -0.19780417
  -0.24547316]]
0.45828942117856203   1Wave Height (m)
0.6221886836370193   1Max Wave Height (m)
0.7165964928478602   2Wave Height (m)
0.8071188206264419   2Max Wave Height (m)
0.826515586009

# Wrapper - Recursive Feature Elimination (RFE) 
https://blog.csdn.net/weixin_48135624/article/details/114296938

In [51]:
from sklearn.feature_selection import RFE #递归特征消除
from sklearn.ensemble import RandomForestClassifier #随机森林
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import load_iris

t0 =time.time()
gbr = GradientBoostingRegressor()

rfe = RFE(estimator=gbr, n_features_to_select=5) #rfe是递归特征处理问题的方法。
#estimator=rf传入用的估计器是什么.
#n_features_to_select,最终剩下几个数据，2剩下两个数据，跑两次。
X_rfe = rfe.fit_transform(X,y)
# GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls').fit(X_train, y_train)
print(time.time()-t0)
X_rfe.shape


220.1814661026001


(175200, 5)

In [52]:
list = []
for i in X_rfe[0]:
    for j in range(len(X[0])):
        if i == X[0][j]:
            list.append(table.columns[j])
list

['1Wave Height (m)',
 '1Tz (s)',
 '1Spread (degrees)',
 '2Max Wave Height (m)',
 '2Tz (s)']

# Embeded

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import MultiTaskElasticNetCV

ElasticNet

In [125]:
# EN
# EN = ElasticNet(alpha=1.0, *, l1_ratio=0.5, fit_intercept=True, 
#                 normalize=False, precompute=False, max_iter=1000, 
#                 copy_X=True, tol=0.0001, warm_start=False, 
#                 positive=False, random_state=0, selection='cyclic')
t0 = time.time()
EN = ElasticNet(random_state=0,tol=0.01).fit(X,y)
model = SelectFromModel(EN, prefit=True)
X_embed_1 = model.transform(X)
print(time.time()-t0)
X_embed_1.shape

0.13951396942138672


(175188, 2)

In [126]:
list = []
for i in X_embed_1[0]:
    for j in range(len(X[0])):
        if i == X[0][j]:
            list.append(table.columns[j])
list

['3Max Wave Height (m)', '3Spread (degrees)']

ElasticNetCV

In [205]:
# ENCV
# ElasticNetCV(*, l1_ratio=0.5, eps=0.001, 
# n_alphas=100, alphas=None, fit_intercept=True, 
# normalize=False, precompute='auto', max_iter=1000, 
# tol=0.0001, cv=None, copy_X=True, verbose=0, n_jobs=None, 
# positive=False, random_state=None, selection='cyclic')[source]

t0 = time.time()
ENCV = ElasticNetCV(cv=5, random_state=0).fit(X,y)
model = SelectFromModel(ENCV, prefit=True)
X_embed_2 = model.transform(X)
print(time.time()-t0)
X_embed_2.shape

1.879143238067627


(175188, 3)

In [206]:
import random
rn = random.randint(1,len(table))
list = []
for i in X_embed_2[rn]:
    for j in range(len(X[rn])):
        if i == X[rn][j]:
            list.append(table.columns[j])
list

['1Wave Height (m)', '1Tz (s)', '2Tz (s)']

Random Forest

In [140]:
# RF
# RandomForestRegressor(n_estimators=100, *, criterion='mse', 
# max_depth=None, min_samples_split=2, min_samples_leaf=1, 
# min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
# min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, 
# oob_score=False, n_jobs=None, random_state=None, verbose=0, 
# warm_start=False, ccp_alpha=0.0, max_samples=None)

t0 = time.time()
RF = RandomForestRegressor(random_state=0).fit(X,y)
model = SelectFromModel(RF, prefit=True)
X_embed_3 = model.transform(X)
print(time.time()-t0)
X_embed_3.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


26.965364456176758


(175176, 2)

In [181]:
list = []
for i in X_embed_3[0]:
    for j in range(len(X[0])):
        if i == X[0][j]:
            list.append(table.columns[j])
list

['3Wave Height (m)', '3Max Wave Height (m)']

MultiTaskElasticNetCV

In [149]:
def split_series(series, n_past, n_future):
    X, y = [],[]
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
    # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

In [173]:
X, y = split_series(table.values,24,12)[0][:,0,:], split_series(table.values,24,12)[1][:,:,7]

In [180]:
# MENCV
# MultiTaskElasticNetCV(*, l1_ratio=0.5, eps=0.001, n_alphas=100, 
# alphas=None, fit_intercept=True, normalize=False, max_iter=1000, 
# tol=0.0001, cv=None, copy_X=True, verbose=0, n_jobs=None, 
# random_state=None, selection='cyclic')

t0 = time.time()
clf = MultiTaskElasticNetCV(cv=5).fit(X,y)
model = SelectFromModel(RF, prefit=True)
X_embed_4 = model.transform(X)
print(time.time()-t0)
X_embed_4.shape

1007.0889339447021


(175165, 2)

In [182]:
list = []
for i in X_embed_4[0]:
    for j in range(len(X[0])):
        if i == X[0][j]:
            list.append(table.columns[j])
list

['3Wave Height (m)', '3Max Wave Height (m)']